In [0]:
import scandir
import os, sys
import rpy2
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import rpy2.robjects as ro
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import dill
import random
import vcf
from hdfstorehelper import HDFStoreHelper
import statsmodels.api as sm
import statsmodels.formula.api as smf
import operator
import traceback
%load_ext rpy2.ipython
from rpy2.robjects import pandas2ri as p2r
p2r.activate()
r = ro.r
import shutil
from utils import read_df, save_df
from pathlib import Path, PurePath
from ipyparallel import Client
from collections import Counter, defaultdict, namedtuple, OrderedDict
from scipy.stats import mannwhitneyu, ks_2samp, f_oneway
import tables
import ujson
import pickle

In [0]:
analysis_dir = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/beagle40/"

In [0]:
z12_df = read_df(analysis_dir, "z12_df")

In [0]:
z12_df.head()

In [0]:
snpmat = z12_df.ix[:,:-1]

In [0]:
locus_names = z12_df.columns[:-1]

In [0]:
pop_names = z12_df['population']

In [0]:
%R -i pop_names -i locus_names -i analysis_dir

In [0]:
snpmat.to_csv(os.path.join(analysis_dir, "snpmat"), sep="\t", header=False, index=False)

In [0]:
%%R
library(OutFLANK)
library(data.table)

In [0]:
%%R
pop_names = unlist(pop_names, use.names=F)
locus_names = unlist(locus_names, use.names=F)
f = paste(analysis_dir, "snpmat", sep="")
snpmat = fread(f, header=F, data.table=F)

In [0]:
%%R
fstdata = MakeDiploidFSTMat(snpmat,locus_names,pop_names)

In [0]:
%%R
NumberOfSamples = 6
outflank_res = OutFLANK(fstdata, 
                        LeftTrimFraction=0.05, 
                        RightTrimFraction=0.05, 
                        Hmin=0.1, 
                        NumberOfSamples, 
                        qthreshold=0.05)

In [0]:
%%R
saveRDS(outflank_res, paste(analysis_dir, "outflank_res.Rdata", sep=""))

In [0]:
$$R


In [0]:
%%R
OutFLANKResultsPlotter(outflank_res,
                       withOutliers=TRUE, 
                       NoCorr=TRUE, 
                       Hmin=0.1, 
                       binwidth=0.005, 
                       Zoom=FALSE,
                       RightZoomFraction=0.05, 
                       titletext=NULL)

In [0]:
%%R
outflank_res$numberHighFstOutliers